In [99]:
import pandas as pd

In [110]:
mailing_group = '20250820-proximos-pasos'

In [100]:
filename = 'BDD_InvitadxsACI_2025.xlsx'
df_internacionales = pd.read_excel(filename, sheet_name='Internacionales')
df_colombia = pd.read_excel(filename, sheet_name='Colombia')
df_bogota = pd.read_excel(filename, sheet_name='Bogotá')

In [101]:
df_internacionales['lista'] = 'Internacionales'
df_colombia['lista'] = 'Colombia'
df_bogota['lista'] = 'Bogotá'

In [102]:
df_internacionales = df_internacionales[df_internacionales['Estado de Carta de Invitación'] == 'Enviada']
df_colombia = df_colombia[df_colombia['Estado de confirmación'] == 'Confirma asistencia']
df_bogota = df_bogota[df_bogota['Estado de confirmación'] == 'Confirma asistencia']

In [103]:
columnas = ['ID', 'Nombre completo', 'Email', 'lista']
df_internacionales = df_internacionales[columnas]
df_colombia = df_colombia[columnas]
df_bogota = df_bogota[columnas]

In [104]:
df_invitados = df_internacionales.merge(df_colombia, on=['ID', 'Nombre completo', 'Email', 'lista'], how='outer')
df_invitados = df_invitados.merge(df_bogota, on=['ID', 'Nombre completo', 'Email', 'lista'], how='outer')

In [105]:
# Quitar los que tiene Email vacío
df_invitados = df_invitados[df_invitados['Email'].notna()]

In [106]:
# Duplicar las filas para los invitados que tienen más de un Email, separados por coma#
# Cuando un invitado tiene más de un email en la columna Email, se deben crear filas adicionales para cada email

df_invitados['Email'] = df_invitados['Email'].str.split(',')
df_invitados = df_invitados.explode('Email')
# Quitar espacios en blanco en la columna Email
df_invitados['Email'] = df_invitados['Email'].str.strip()


In [111]:
# Idenfificar los invidados a los que ya se les envió mensaje desde la hoja mailing
df_mailing = pd.read_excel(filename, sheet_name='mailing')

# Quitar de df_invitados los emails que aparecene en df_mailing['email']
df_invitados = df_invitados[~df_invitados['Email'].isin(df_mailing['email'])]

df_invitados['envio'] = mailing_group

df_invitados.shape

(14, 5)

In [114]:
with pd.ExcelWriter('users_upload.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_invitados.to_excel(writer, sheet_name='mailing', index=False) 